### **Nama : Mitslina**
### **NIM : 24060121130068**
### **2. Buatlah evaluasi algoritma dengan dataset yang telah dicoba pada tugas praktikum sebelumnya (dengan menggunakan 3 model yaitu KNN, NB dan SVM)**
### **Data set : Dry Bean Dataset**

In [1]:
# import file gdrive lokal
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# import library pandas
import pandas as pd

In [3]:
# menuliskan path gdrive lokal yang sesuai
path = "/content/gdrive/MyDrive/PrakPM/Dry_Bean_Dataset.xlsx"
# memberikan nama dari fitur (kolom) yang akan digunakan
names = ['area', 'perimeter', 'majorAxisLength', 'minorAxisLength', 'aspectRation', 'eccentricity', 'convexArea', 'equivDiameter', 'extent', 'solidity', 'roundness', 'compactness', 'shapeFactor1', 'shapeFactor2', 'shapeFactor3', 'shapeFactor4', 'class']
# memuat dataset excel sesuai variabel path dan names
dataset = pd.read_excel(path, names=names)

In [4]:
# melihat isi dataset
dataset.head(5)

,area,perimeter,majorAxisLength,minorAxisLength,aspectRation,eccentricity,convexArea,equivDiameter,extent,solidity,roundness,compactness,shapeFactor1,shapeFactor2,shapeFactor3,shapeFactor4,class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [5]:
# melihat dimensi dataset
dataset.shape

(13611, 17)

### **Untuk memperkirakan akurasi pada data yang tidak diketahui sebelumnya terdapat beberapa data yang dapat dilakukan :**

### **I. Membuat Validasi Dataset**

In [6]:
# import library berikut untuk menggunakan fungsi split data training (train_test_split())
from sklearn import model_selection

In [7]:
# #persiapan data
array = dataset.values
# X menyimpan inputan fitur model machine learning yaitu fitur index-0 -- index-15
X = array[:,0:16]
# Y berisi target data, index ke 16 yaitu 'class'
Y = array[:,16]


# #pembagian data
# validasi data model 20% dari total data keseluruhan, sehingga 80% adalah data training
validation_size = 0.20
# untuk mengacak data yang akan dipakai untuk split data, utk memastikan hasil  pembagian data konsisten ketika menjalankan kode berulang kali
seed = 7
# X dan Y sesuai penjelasan yang telah disebutkan di atas, train untuk data training dan validation untuk data validasi
# model_selection.train_test_split untuk membagi dataset menjadi dua bagian yaitu data latih (training set) dan data validasi (validation set)
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

### **Penjelasan:**
Berdasarkan kode tersebut kita telah membagi dataset menjadi dua bagian utama:

*   X_train dan Y_train untuk melatih model.
*   X_validation dan Y_validation untuk menguji dan mengevaluasi model.

Dengan cara ini, kita dapat menggunakan X_train dan Y_train untuk melatih model, dan kemudian menggunakan X_validation dan Y_validation untuk mengukur sejauh mana model tersebut dapat melakukan prediksi yang akurat pada data yang belum pernah dilihat sebelumnya (data validasi).

In [8]:
# melihat hasil dimensinya (record, fitur)
print("X_train :", X_train.shape)
print("X_validation :", X_validation.shape)
print("Y_train :", Y_train.shape)
print("Y_validation :", Y_validation.shape)

X_train : (10888, 16)
X_validation : (2723, 16)
Y_train : (10888,)
Y_validation : (2723,)


Penjelasan:
* 80% data training : 10888
* 20% data validasi : 2723

In [9]:
# distribusi kelas data
dataset.groupby('class').size()

class
BARBUNYA    1322
BOMBAY       522
CALI        1630
DERMASON    3546
HOROZ       1928
SEKER       2027
SIRA        2636
dtype: int64

Penjelasan:

Berdasarkan distribusi kelas tersebut maka kasus tersebut merupakan kasus dengan ketidakseimbangan kelas sehingga lebih baik menggunakan F1-score. F1-Score mempertimbangkan baik precision (ketepatan) dan recall (sensitivitas atau true positive rate). Ini membuatnya lebih baik daripada accuracy ketika kita memiliki ketidakseimbangan kelas, di mana satu kelas mungkin dominan.

### **II. K-Folds Cross Validation**

Kita akan menggunakan validasi silang 10 kali lipat untuk memperkirakan akurasi. Untuk itu dataset dibagi menjadi 10 bagian, 9 untuk latihan dan 1 untuk pengujian dan ulangi untuk semua kombinasi.

In [10]:
# training split --> memisahkan data menjadi 2 bagian, iterasi sekali
# K fold --> memisahkan data menjadi beberapa K fold, setiap fold secara bergantian dijadikan sebagai set pengujian sedangkan yang lainnya digunakan sebagai pelatihan

# Test options and evaluation metric menggunakan F1-Score dengan weighted averaging
seed = 7
scoring = 'f1_weighted'

### **III. Membangun Model**

In [11]:
# import library berisi algoritma klasifikasi yang akan digunakan
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Spot Check Algorithms. Evaluasi model menggunakan cross validation
models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

### **Penjelasan:**
Pada kode tersebut, akan mengevaluasi dengan beberapa algoritma untuk mengetahui algoritma yang cocok dengan studi kasus ini. Model yang digunakan yaitu K-Nearest Neighbors (KNN),
Gaussian Naive Bayes (NB) dan Support Vector Machines (SVM).

In [12]:
# evaluate each model in turn

# menyimpan hasil evaluasi K fold validation dari masing-masing model
results = []
# menyimpan nama model
names = []

for name, model in models:
  # membagi data menjadi training dan validation menggunakan kfold validation, dengan k atau n_splits nya adalah 10 dan random_state = 7
  # mengaktifkan shuffle=True. shuffle digunakan untuk menentukan apakah data harus diacak atau tidak sebelum dibagi
  kfold = model_selection.KFold(n_splits=10, random_state=seed, shuffle=True)
  # model_selection.cross_val_score untuk menghitung nilai scoring dari setiap iterasi
  # mengukur kinerja dari data dari model yang sudah didefinisikan dengan data latih yang sudah di split (X_train dan Y_train), cv = kfold yg sudah didefinisikan, scoring = f1_weighted
  cv_results = model_selection.cross_val_score(model, X_train, Y_train,  cv=kfold, scoring=scoring)
  # memasukan cv_results ke dalam result untuk setiap iterasi
  results.append(cv_results)
  # memasukan name ke dalam names untuk setiap iterasi
  names.append(name)
  # menampilkan pesan nama model, rata2 final, dan stdev final
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

KNN: 0.720164 (0.009220)
NB: 0.761147 (0.007715)
SVM: 0.599921 (0.006756)


### **Penjelasan:**
*   Model KNN menghasilkan final rata-rata=0.720164 dan stdev=0.009220
*   Model NB menghasilkan final rata-rata=0.761147  dan stdev=0.007715
*   Model SVM menghasilkan final rata-rata=0.599921 dan stdev=0.006756

### **IV. Memilih Model Terbaik**

### **Penjelasan:**
Berdasarkan hasil output tersebut di atas, kita dapat melihat bahwa Naive Bayes (NB) memiliki nilai akurasi perkiraan terbesar. Setelah mengetahui model yang paling akurat yaitu NB, selanjutnya kita dapat mencoba melakukan pengujian tentang keakuratan model NB terhadap data yang ada.

In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [15]:
# Membuat prediksi validasi dataset

NB = GaussianNB()
# melatih (fit) model NB menggunakan data latih
NB.fit(X_train, Y_train)
# menguji data NB yang dihasilkan dari proses pelatihan data dengan testing data
predictions = NB.predict(X_validation)

# menampilkan hasil
print('accuracy score: ', accuracy_score(Y_validation, predictions))
print('confusion matrix: \n', confusion_matrix(Y_validation, predictions))
print('\n\t\t classification report: \n', classification_report(Y_validation, predictions))

accuracy score:  0.7598237238340066
confusion matrix: 
 [[134   0 113   0  21   0   8]
 [  0  90   0   0   0   0   0]
 [ 44   1 275   0  17   0   2]
 [  0   0   0 613   0  90  38]
 [ 13   0  18  14 296   0  42]
 [  3   0   0  47   7 282  50]
 [  0   0   0  29  50  47 379]]

		 classification report: 
               precision    recall  f1-score   support

    BARBUNYA       0.69      0.49      0.57       276
      BOMBAY       0.99      1.00      0.99        90
        CALI       0.68      0.81      0.74       339
    DERMASON       0.87      0.83      0.85       741
       HOROZ       0.76      0.77      0.76       383
       SEKER       0.67      0.72      0.70       389
        SIRA       0.73      0.75      0.74       505

    accuracy                           0.76      2723
   macro avg       0.77      0.77      0.77      2723
weighted avg       0.76      0.76      0.76      2723



### **Penjelasan:**
Output yang dihasilkan dari evaluasi model memberikan gambaran tentang seberapa baik model Naive Bayes (NB) dapat melakukan klasifikasi pada data validasi.

- Accuracy Score:
Akurasi adalah rasio prediksi yang benar (positif dan negatif) terhadap total data. Dalam konteks ini, akurasi sebesar 0.76 (76%) menunjukkan bahwa model NB dapat mengklasifikasikan data dengan benar sekitar 76% pada saat data validasi.

- Confusion Matrix:
Matriks konfusi adalah tabel yang menunjukkan jumlah prediksi yang benar dan salah untuk setiap kelas. Dalam kasus ini, matriks konfusi memiliki tujuh baris dan tujuh kolom, yang sesuai dengan tiga kelas Iris (BARBUNYA, BOMBAY, CALI, DERMASON, HOROZ, SEKER, SIRA).
 - Baris dan kolom mewakili kelas, dan setiap elemen menyatakan jumlah prediksi untuk kelas tertentu. Sebagai contoh, pada baris kedua (BOMBAY) dan kolom kedua (BOMBAY), terdapat 90 prediksi yang benar.

- Classification Report:
Laporan klasifikasi memberikan matriks precision,  recall, f1-score untuk setiap kelas, serta rata-rata dari matriks tersebut untuk semua kelas (macro avg dan weighted avg).
 - Precision: Seberapa akurat model dalam mengklasifikasikan positif. Misalnya, presisi untuk BARBUNYA adalah 0.69, yang berarti 69% dari semua prediksi positif untuk BARBUNYA adalah benar.
 - Recall: Seberapa baik model dapat menangkap semua instance dari suatu kelas. Misalnya, recall untuk BARBUNYA adalah 0.49, yang berarti model berhasil menangkap 49% dari semua instance BARBUNYA.
 - F1-score: Rata-rata dari precision dan recall.
 - Support: Jumlah instance yang sebenarnya dari setiap kelas.

Berdasarkan hasil yang diberikan, model memiliki kinerja baik pada beberapa kelas seperti BOMBAY dengan presisi, recall, dan F1-score sangat tinggi, sedangkan untuk beberapa kelas lainnya, kinerja model mungkin tidak sebaik kelas BOMBAY.